In [1]:
import torch
import wandb
import pprint
from tqdm.notebook import tqdm
import numpy as np

In [2]:
import os
os.environ["http_proxy"]="http://proxy61.iitd.ac.in:3128"
os.environ["https_proxy"]="http://proxy61.iitd.ac.in:3128"


In [4]:
track_columns = ['new node test average_precision',
 'train roc_auc',
 'new node val first_3_roc_auc',
 'new node test first_3_roc_auc',
 'new node test first_3_average_precision',
 'new node val average_precision',
 'new node val first_1_average_precision',
 'new node test first_1_average_precision',
 'new node test first_1_roc_auc',
 'new node test first_10_roc_auc',
 'new node val first_3_average_precision',
 'val average_precision',
 'val roc_auc',
 'new node val roc_auc',
 'new node val first_10_average_precision',
 'new node test roc_auc',
 'train average_precision',
 'test roc_auc',
 'new node val first_1_roc_auc',
 'new node val first_10_roc_auc',
 'test average_precision',
 'new node test first_10_average_precision']
import pandas as pd 
import wandb
api = wandb.Api()

runs = api.runs("fb-graph-proj/fb-graph-proj-dyglib")

for cnt, run in enumerate(tqdm(runs)):
    # Find epoch with best avg precision
    if cnt < 10:
        val_key = 'val average_precision'
        hist = run.history(samples = 200, keys = [val_key])[val_key]
        best_epoch_num_test = np.argmax(hist[9::10])
        best_epoch_num = np.argmax(hist)
    else:
        val_key = 'val_loss'
        hist = run.history(samples = 200, keys = [val_key])[val_key]
        best_epoch_num_test = np.argmin(hist[9::10])
        best_epoch_num = np.argmin(hist)
    print(best_epoch_num)
    for key in track_columns:
        hist = run.history(samples = 200, keys = [key])[key]
        print(f"From {run.summary[key]} to ", end ="")
        if 'test' in key:
            run.summary[key] = hist[best_epoch_num_test]
            print(f"{run.summary[key]}")
        else:
            run.summary[key] = hist[best_epoch_num]
            print(f"{run.summary[key]}")
    run.summary.update()

  0%|          | 0/126 [00:00<?, ?it/s]

0
From 0.8672813840062237 to 0.8672813840062237
From 0.8882860385008542 to 0.8882860385008542
From 0.8398459762319771 to 0.8398459762319771
From 0.8610131644859583 to 0.8610131644859583
From 0.8850942211354055 to 0.8850942211354055
From 0.8799009856160099 to 0.8799009856160099
From 0.8838137732962628 to 0.8838137732962628
From 0.8829440288708276 to 0.8829440288708276
From 0.851919985337518 to 0.851919985337518
From 0.8348445829975917 to 0.8348445829975917
From 0.8654948792328366 to 0.8654948792328366
From 0.9363782335898408 to 0.9363782335898408
From 0.9321759013304018 to 0.9321759013304018
From 0.8552387538841385 to 0.8552387538841385
From 0.8716471408107201 to 0.8716471408107201
From 0.8423322603338524 to 0.8423322603338524
From 0.8986427270894599 to 0.8986427270894599
From 0.9174959425571696 to 0.9174959425571697
From 0.8574043804506223 to 0.8574043804506223
From 0.8474606051140972 to 0.8474606051140972
From 0.9147969136770786 to 0.9147969136770786
From 0.8653873255942753 to 0.86538

KeyError: 'new node val first_3_roc_auc'

In [61]:
def get_average_over_last_five_epochs(run_id):
  # Takes wandb run id and returns average of last 5 readings of the following metrics
  # new node test average_precision
  # test average_precision
  # test roc_auc
  # new node test roc_auc
  api = wandb.Api()
  run = api.run(f"/fb-graph-proj/fb-graph-proj-dyglib/runs/{run_id}")
  index = torch.argmax(torch.tensor(run.history()["val average_precision"].tolist()))
  METRICS_LIST = ["new node test average_precision", "test average_precision", "test roc_auc", "new node test roc_auc"]
  avg_metric = {}
  for metric in METRICS_LIST:
    avg_metric[metric] = round(float(100 * torch.nanmean(torch.tensor(run.history()[metric].tolist()))), 2)
  print(run.config["dataset"])
  pprint.pprint(avg_metric)
  # return avg_metric

In [70]:
get_average_over_last_five_epochs("p0ft9874")

Contacts
{'new node test average_precision': 85.31,
 'new node test roc_auc': 90.1,
 'test average_precision': 80.5,
 'test roc_auc': 87.03}
